In [14]:
import pandas as pd
import time
import requests
import os
import sys
import random

# ---------------------------------------------
# CONFIGURATION - Change these values as needed
# ---------------------------------------------
# API details
API_KEY = "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"
API_URL = "https://openrouter.ai/api/v1/chat/completions"
MODEL_NAME = "moonshotai/kimi-dev-72b:free"

# Input/Output file names
input_excel_path = 'dataset - Copy.xlsx'
output_excel_path = 'kimi_dev_72b_results.xlsx'

# Sheet to process
sheet_to_process = 'allenai lila _ 100'

# How often to save progress (e.g., every 5 rows)
checkpoint_every = 5

# Delay between *successful* API calls in seconds
sleep_time = 1.5

# --- (NEW) Retry Configuration for API Failures ---
MAX_RETRIES = 4  # Max number of retries for a single request
INITIAL_RETRY_DELAY = 5  # Initial delay in seconds for the first retry
# ---------------------------------------------

# --- Custom Exception for Rate Limiting ---
class RateLimitException(Exception):
    """Custom exception for handling API rate limit errors."""
    pass

# --- (MODIFIED) Function to call LLM API with Retry Logic ---
def call_llm_api(prompt):
    """
    Sends a prompt to the LLM API and returns the response.
    Includes retry logic with exponential backoff for transient errors.
    """
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {API_KEY}"
    }
    data = {
        "model": MODEL_NAME,
        "messages": [
            {"role": "system", "content": "You are a helpful assistant designed to solve math problems. Provide a clear, step-by-step solution and state the final answer."},
            {"role": "user", "content": prompt}
        ]
    }
    
    last_error = None
    for attempt in range(MAX_RETRIES):
        start_time = time.time()
        try:
            response = requests.post(API_URL, headers=headers, json=data, timeout=90)
            latency = time.time() - start_time

            # --- SUCCESS ---
            if response.status_code == 200:
                res_json = response.json()
                answer = res_json['choices'][0]['message']['content']
                usage = res_json.get('usage', {})
                return {
                    "answer": answer,
                    "input_tokens": usage.get('prompt_tokens', 0),
                    "output_tokens": usage.get('completion_tokens', 0),
                    "total_tokens": usage.get('total_tokens', 0),
                    "latency": latency
                }

            # --- FATAL: RATE LIMITING ---
            elif response.status_code == 429:
                error_details = response.json().get('error', {}).get('message', 'No details provided.')
                raise RateLimitException(f"API rate limit reached. Details: '{error_details}'")

            # --- RETRYABLE ERRORS (Server-side issues) ---
            elif response.status_code in [408, 500, 502, 503, 504]:
                error_text = f"API Error (Code {response.status_code}): {response.reason}"
                print(f"   ⚠️  {error_text}. Will retry...")
                last_error = {"answer": error_text, "input_tokens": 0, "output_tokens": 0, "total_tokens": 0, "latency": latency}

            # --- FATAL ERRORS (Client-side issues like bad auth or invalid request) ---
            else:
                error_text = response.json().get('error', {}).get('message', str(response.text))
                final_error = f"API Error (Code {response.status_code}): {error_text}"
                print(f"   ❌ Unrecoverable {final_error}")
                return {"answer": final_error, "input_tokens": 0, "output_tokens": 0, "total_tokens": 0, "latency": latency}

        except requests.exceptions.RequestException as e:
            latency = time.time() - start_time
            print(f"   ⚠️  Network Error: {e}. Will retry...")
            last_error = {"answer": f"Network Error: {e}", "input_tokens": 0, "output_tokens": 0, "total_tokens": 0, "latency": latency}
        
        except requests.exceptions.JSONDecodeError as e:
            latency = time.time() - start_time
            print(f"   ⚠️  JSON Parse Error: {e}. Malformed response. Will retry...")
            last_error = {"answer": f"JSON Parse Error: {response.text}", "input_tokens": 0, "output_tokens": 0, "total_tokens": 0, "latency": latency}

        # --- Exponential Backoff ---
        # Don't sleep on the last attempt
        if attempt < MAX_RETRIES - 1:
            # e.g., 5s, 10s, 20s... + some randomness
            backoff_time = INITIAL_RETRY_DELAY * (2 ** attempt) + random.uniform(0, 1)
            print(f"      -> Retrying in {backoff_time:.2f} seconds... (Attempt {attempt + 2}/{MAX_RETRIES})")
            time.sleep(backoff_time)
        else:
            print(f"   ❌ Max retries reached. Failing permanently for this row.")

    # If all retries fail, return the last captured error
    return last_error

# ---------------------------------------------
# Main Processing Logic (No changes needed here)
# ---------------------------------------------
print("--- Starting Script ---")

# --- Create output directory if it doesn't exist ---
output_dir = os.path.dirname(output_excel_path)
if output_dir:
    os.makedirs(output_dir, exist_ok=True)
    print(f"✅ Ensured output directory exists: '{output_dir}'")

# 1. Load input Excel file
try:
    xls = pd.ExcelFile(input_excel_path)
    print(f"✅ Successfully loaded input file: '{input_excel_path}'")
except FileNotFoundError:
    print(f"❌ FATAL ERROR: Input file not found at '{input_excel_path}'. Exiting.")
    sys.exit()

# 2. Check for existing output file to enable resuming
output_xls = None
if os.path.exists(output_excel_path):
    try:
        output_xls = pd.ExcelFile(output_excel_path)
        print(f"ℹ️  Found existing output file. Will resume progress.")
    except Exception as e:
        print(f"⚠️ Warning: Could not read existing output file '{output_excel_path}'. It might be corrupted. A new file will be created. Error: {e}")

# 3. Set up the ExcelWriter to save progress
try:
    writer_mode = 'a' if output_xls else 'w'
    writer_kwargs = {
        "engine": 'openpyxl',
        "mode": writer_mode
    }
    if writer_mode == 'a':
        writer_kwargs['if_sheet_exists'] = 'replace'

    with pd.ExcelWriter(output_excel_path, **writer_kwargs) as writer:
        print(f"\n🔄 Processing sheet: '{sheet_to_process}'")

        if sheet_to_process not in xls.sheet_names:
            print(f"❌ FATAL ERROR: Sheet '{sheet_to_process}' not found in the input file. Exiting.")
            sys.exit()

        df_to_process = pd.read_excel(xls, sheet_name=sheet_to_process)

        # RESUME LOGIC
        result_columns = ['Result Answer from LLM', 'Input Tokens', 'Output Tokens', 'Total Tokens', 'Latency (s)']
        if output_xls and sheet_to_process in output_xls.sheet_names:
            df_existing_output = pd.read_excel(output_xls, sheet_name=sheet_to_process)
            print("   -> Merging existing results...")
            for col in result_columns:
                if col in df_existing_output.columns:
                    df_to_process[col] = df_existing_output.get(col)
                else:
                    df_to_process[col] = None
        else:
            for col in result_columns:
                df_to_process[col] = None

        df_to_process['Result Answer from LLM'] = df_to_process['Result Answer from LLM'].astype('object')

        # --- Main Processing Loop ---
        try:
            total_rows = len(df_to_process)
            for idx, row in df_to_process.iterrows():
                if pd.notna(row.get('Result Answer from LLM')) and row.get('Result Answer from LLM') != "":
                    print(f"➡️  Row {idx+1}/{total_rows} already processed. Skipping.")
                    continue

                question = row.get('Questions', 'No question found in row')
                options = row.get('Options', '')

                if pd.isna(options) or options == '':
                    prompt = f"Solve this math problem: {question}"
                else:
                    prompt = f"Solve this math problem: {question}\nOptions: {options}"

                print(f"⚙️  Processing row {idx+1}/{total_rows}...")
                result = call_llm_api(prompt)

                df_to_process.at[idx, 'Result Answer from LLM'] = result['answer']
                df_to_process.at[idx, 'Input Tokens'] = result['input_tokens']
                df_to_process.at[idx, 'Output Tokens'] = result['output_tokens']
                df_to_process.at[idx, 'Total Tokens'] = result['total_tokens']
                df_to_process.at[idx, 'Latency (s)'] = round(result['latency'], 2)

                print(f"✅ Processed row {idx+1}/{total_rows} | Latency: {round(result['latency'], 2)}s")

                time.sleep(sleep_time)

                if (idx + 1) % checkpoint_every == 0 or (idx + 1) == total_rows:
                    df_to_process.to_excel(writer, sheet_name=sheet_to_process, index=False)
                    print(f"💾 Checkpoint saved at row {idx+1}")

        except RateLimitException as e:
            print("\n" + "="*60)
            print("🛑 EXECUTION HALTED: API RATE LIMIT REACHED")
            print(f"   Error Details: {e}")
            print("   Progress up to this point will be saved.")
            print("="*60)
        except Exception as e:
            print(f"\nAn unexpected error occurred during processing: {e}")
            print("Saving progress before exiting.")

        finally:
            print("\nSaving final progress...")
            df_to_process.to_excel(writer, sheet_name=sheet_to_process, index=False)
            print(f"✅ Final data for sheet '{sheet_to_process}' saved.")

except Exception as e:
    print(f"\n❌ A critical error occurred with the Excel file writer: {e}")
    print("   Please ensure the output file is not open in another program and that you have permission to write to the folder.")


print(f"\n🎉 All operations finished. Results are saved in '{output_excel_path}'")

--- Starting Script ---
✅ Loaded input file: dataset - Copy.xlsx
ℹ️ Existing output file found. Will resume progress.
➡️  Row 1/100 already processed. Skipping.
➡️  Row 2/100 already processed. Skipping.
➡️  Row 3/100 already processed. Skipping.
➡️  Row 4/100 already processed. Skipping.
➡️  Row 5/100 already processed. Skipping.
➡️  Row 6/100 already processed. Skipping.
➡️  Row 7/100 already processed. Skipping.
➡️  Row 8/100 already processed. Skipping.
➡️  Row 9/100 already processed. Skipping.
➡️  Row 10/100 already processed. Skipping.
⚙️  Processing row 11/100...


RateLimitException: API rate limit reached. Details: 'Rate limit exceeded: free-models-per-day. Add 10 credits to unlock 1000 free model requests per day'